# Comparing performance of belief maximization algorithm with and without observabilities

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)

<module 'source.players.belief_max' from '..\\source\\players\\belief_max.py'>

### Creating configurations to save into a csv file

In [7]:
time_horizon = 1000
T = 5
#targets = util.gen_targets(T, low=1, high=10)
targets = [5, 3, 8, 4, 9]
values = tuple((v, v) for v in targets)
#obs = gen_observabilities_correlated_with_values(values)
obs = {0: 0.75, 1: 0.75, 2: 0.75, 3: 0.75, 4: 0.75}
obs_vect = [obs.get(i) for i in range(len(targets))]
distributions = []
distributions.append(tuple([0.2, 0.2, 0.2, 0.2, 0.2]))
distributions.append(tuple([0.9, 0.1, 0, 0, 0]))
distributions.append(tuple([0.9, 0, 0.1, 0, 0]))
distributions.append(tuple([0.9, 0, 0, 0.1, 0]))
distributions.append(tuple([0.9, 0, 0, 0, 0.1]))
distributions.append(tuple([0.9, 0.025, 0.025, 0.025, 0.025]))
distributions.append(tuple([0.025, 0.9, 0.025, 0.025, 0.025]))
distributions.append(tuple([0.025, 0.025, 0.9, 0.025, 0.025]))
distributions.append(tuple([0.025, 0.025, 0.025, 0.9, 0.025]))
distributions.append(tuple([0.025, 0.025, 0.025, 0.025, 0.9]))
distributions.append(tuple([1, 0, 0, 0, 0]))
distributions.append(tuple([0, 1, 0, 0, 0]))
distributions.append(tuple([0, 0, 1, 0, 0]))
distributions.append(tuple([0, 0, 0, 1, 0]))
distributions.append(tuple([0, 0, 0, 0, 1]))
distributions.append(tuple([0.92, 0, 0.07, 0, 0.01]))
for i in range(10):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)

[(0.2, 0.2, 0.2, 0.2, 0.2), (0.9, 0.1, 0, 0, 0), (0.9, 0, 0.1, 0, 0), (0.9, 0, 0, 0.1, 0), (0.9, 0, 0, 0, 0.1), (0.9, 0.025, 0.025, 0.025, 0.025), (0.025, 0.9, 0.025, 0.025, 0.025), (0.025, 0.025, 0.9, 0.025, 0.025), (0.025, 0.025, 0.025, 0.9, 0.025), (0.025, 0.025, 0.025, 0.025, 0.9), (1, 0, 0, 0, 0), (0, 1, 0, 0, 0), (0, 0, 1, 0, 0), (0, 0, 0, 1, 0), (0, 0, 0, 0, 1), (0.92, 0, 0.07, 0, 0.01), (0.036, 0.089, 0.767, 0.083, 0.025), (0.434, 0.015, 0.086, 0.205, 0.26), (0.333, 0.109, 0.022, 0.422, 0.114), (0.0, 0.036, 0.017, 0.86, 0.087), (0.049049049049049054, 0.1691691691691692, 0.17117117117117117, 0.10210210210210209, 0.5085085085085085), (0.04095904095904097, 0.11288711288711291, 0.3236763236763237, 0.1038961038961039, 0.4185814185814186), (0.36400000000000005, 0.08700000000000001, 0.40700000000000003, 0.13000000000000003, 0.012000000000000002), (0.215, 0.028, 0.15, 0.092, 0.515), (0.28, 0.255, 0.235, 0.101, 0.129), (0.184, 0.531, 0.217, 0.056, 0.012)]


In [8]:
g1 = game.Game(values, time_horizon)
g1.attackers = [1]
g1.defenders = [0]
a1 = [attackers.StackelbergAttacker(g1, 1)]
d1 = [bm.FB(g1, 0)]
p1 = [attackers.StackelbergAttacker(g1, 1)]
stoc_prof = []
for d in distributions:
    stoc_prof.append(attackers.StochasticAttacker(g1, 1, 1, *d))
att_prof = [attackers.StackelbergAttacker(g1, 1)]
stoc_prof_and_att = stoc_prof + att_prof
g1.set_players(d1, a1, stoc_prof_and_att)

go1 = game.GameWithObservabilities(values, time_horizon)
go1.attackers = [1]
go1.defenders = [0]
a1 = [attackers.StackelbergAttacker(go1, 1)]
d1 = [bm.FB(go1, 0)]
p1 = [attackers.StackelbergAttacker(go1, 1)]
stoc_prof1 = []
for d in distributions:
    stoc_prof1.append(attackers.StochasticAttacker(go1, 1, 1, *d))
att_prof1 = [attackers.StackelbergAttacker(go1, 1)]
stoc_prof_and_att1 = stoc_prof1 + att_prof1
go1.set_players(d1, a1, stoc_prof_and_att1)

In [9]:
def print_header(targets, profiles, observabilities):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "," + ",".join(["Obs" + str(i) for i in range(len(observabilities))]) + "\n" 

In [10]:
def print_row(targets, game_type, time_horizon, d, a, p, obs):
    return ",".join([str(i) for i in ([d+"_vs_"+a+"_"+game_type] + [time_horizon] + targets + [d] + [a] + [x for x in p] + obs) ])+"\n"

In [11]:
(print_header(targets, stoc_prof_and_att, obs_vect))

'Name,T,0,1,2,3,4,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Obs0,Obs1,Obs2,Obs3,Obs4\n'

In [12]:
defender =["FB1", "FB1"]

In [13]:
profiles = []
for p in stoc_prof_and_att:
    profiles.append(util.print_adv(p))
profiles1 = []
for p in stoc_prof_and_att1:
    profiles1.append(util.print_adv(p))

In [14]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Exp3"
batch_name = "belief_max_obs_vs_no_obs"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [15]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, stoc_prof_and_att, obs_vect))
    f.write(print_row(targets, "no_obs", time_horizon,defender[0],"sta1",profiles,[]))
    f.write(print_row(targets, "obs", time_horizon,defender[1],"sta1",profiles1, obs_vect))

In [16]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,Defender,Attacker,Profile,...,Profile.22,Profile.23,Profile.24,Profile.25,Profile.26,Obs0,Obs1,Obs2,Obs3,Obs4
0,FB1_vs_sta1_no_obs,1000,5,3,8,4,9,FB1,sta1,sto1-0.36400000000000005-0.08700000000000001-0...,...,sta1,sto1-0.333-0.109-0.022-0.422-0.114,sto1-0.9-0-0.1-0-0,sto1-0.28-0.255-0.235-0.101-0.129,sto1-0.2-0.2-0.2-0.2-0.2,NaN,NaN,NaN,NaN,NaN
1,FB1_vs_sta1_obs,1000,5,3,8,4,9,FB1,sta1,sto1-0-0-1-0-0,...,sto1-0.9-0.025-0.025-0.025-0.025,sto1-0.2-0.2-0.2-0.2-0.2,sto1-0.9-0-0-0-0.1,sto1-0.049049049049049054-0.1691691691691692-0...,sto1-0.184-0.531-0.217-0.056-0.012,0.75,0.75,0.75,0.75,0.75


### Parsing the csv to generate a batch containing the configurations and running them

In [17]:
#os.mkdir("../NewExperiments/Exp2/belief_max_obs_vs_no_obs")

In [18]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [19]:
b.parse_batch()

In [20]:
batches = []
batches.append(b)

In [21]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=10)

### Collecting results of the runs

In [22]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the two configurations "step-by-step"

In [23]:
gm = deepcopy(batches[0].configurations[0].game)

In [24]:
gm

<Game values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [25]:
e = runner.Experiment(gm)

In [26]:
for i in range(900):
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: (1, 0, 0, 0, 0), 1: [0, 0, 0, 0, 1]}
{1: [4], 0: [0]}
{0: (0, 0, 0, 0, 1), 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0,

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0

In [27]:
e.game

<Game values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [28]:
e.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340925,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.190303301163861,
 <StochasticAttacker id:1 resources:1>: -1.2740214327993638,
 <StochasticAttacker id:1 resources:1>: -0.09126790561131946,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -0.1133238910766163,
 <StochasticAttacker id:1 resources:1>: -3.32124264636016,
 <StochasticAttacker id:1 resources:1>: -1.6956695963863746,
 <Stochasti

In [29]:
gm1 = deepcopy(batches[0].configurations[1].game)

In [30]:
gm1

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [31]:
e1 = runner.Experiment(gm1)

In [32]:
for i in range(900):
    e1.run_interaction_with_observations()
    print(e1.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e1.game.history[-1])
    print(e1.game.observation_history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: (0, 0, 0, 1, 0), 1: [0, 0, 0, 0, 1]}
{1: [4], 0: [3]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: (0, 0, 0, 0, 1), 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}


{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 

In [33]:
e1.game

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [34]:
e1.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: -1.1129739779822934,
 <StochasticAttacker id:1 resources:1>: -3.688879454114005,
 <StochasticAttacker id:1 resources:1>: -3.31473542646758,
 <StochasticAttacker id:1 resources:1>: -3.676243002079115,
 <StochasticAttacker id:1 resources:1>: -0.10840989264820851,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340936,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StackelbergAttacker id:1 resources:1>: 0.0,
 <StochasticAttacker id:1 resources:1>: -3.676243002079115,
 <StochasticAttacker id:1 resources:1>: -0.8422254363670488,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.0222401724074046,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.5353064195098325,
 <StochasticAttacker id:1 resources:1>: -0.13063341972759857,
 <Stochast